In [9]:
%load_ext autoreload
%autoreload 2
import torch
from torch import nn, optim
import numpy as np
import pandas as pd
import scipy
import matplotlib as mpl
import matplotlib.pyplot as plt
import pyro
import pyro.distributions as dist
from pyro.distributions import Categorical, Normal, Multinomial, Binomial, MultivariateNormal, Beta, constraints
from pyro.distributions.torch import Bernoulli
import pyro.infer as infer
from pyro.infer import TraceEnum_ELBO, Trace_ELBO, config_enumerate, SVI
from pyro import poutine

plt.style.use('seaborn-v0_8')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
data = dist.Normal(2, 10.).sample((1000, ))

In [11]:
@config_enumerate
def model(d):
    locs = torch.tensor([2, 4, 5])
    p = torch.tensor([0.3, 0.4, 0.3])
    idx = pyro.sample("idx", dist.Categorical(p))
    with pyro.plate("data", len(data)):
        z = pyro.sample("z", dist.Normal(p[idx], 2.0), obs=d)

def guide(data):
    pass
pyro.set_rng_seed(0)
torch.manual_seed(0)
svi = SVI(model, guide, pyro.optim.Adam({"lr": 1e-2}), TraceEnum_ELBO(max_plate_nesting=1))
print(svi.step(data))


15273.580078125


In [12]:
@config_enumerate
def model(d):
    locs = torch.tensor([2, 4, 5])
    p = torch.tensor([0.3, 0.4, 0.3])
    idx = pyro.sample("idx", dist.Categorical(p))
    with pyro.plate("data", len(data)):
        z = pyro.sample("z", dist.Normal(p[idx], 2.0), obs=d)

def guide(data):
    pass
pyro.set_rng_seed(0)
torch.manual_seed(0)
trace = poutine.trace(poutine.enum(model, first_available_dim=-2)).get_trace(data)

In [13]:
trace.compute_log_prob()

In [14]:
trace.nodes["z"]["value"].size()

torch.Size([1000])

In [15]:
trace.nodes["z"]['log_prob'].size()

torch.Size([3, 1000])

In [16]:
trace.nodes["z"]['log_prob'].sum() / 3

tensor(-15304.9834)